In [2]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style

alt.themes.enable('light')

ThemeRegistry.enable('light')

In [2]:
us_df = pd.read_json("https://api.economicsobservatory.com/usa/eGWh?vega")
us_df['series'] = 'USA'
chn_df = pd.read_json("https://api.economicsobservatory.com/chn/eGWh?vega")
chn_df['series'] = 'China'
gb_df = pd.read_json("https://api.economicsobservatory.com/gbr/eGWh?vega")
gb_df['series'] = 'UK'

df = pd.concat([us_df, chn_df, gb_df])
df

,date,value,series
0,1971-01-01,1703380.000,USA
1,1972-01-01,1851109.000,USA
2,1973-01-01,1965509.000,USA
3,1974-01-01,1957335.000,USA
4,1975-01-01,2011160.000,USA
...,...,...,...
47,2018-01-01,329499.038,UK
48,2019-01-01,321442.302,UK
49,2020-01-01,309209.526,UK
50,2021-01-01,305422.446,UK


In [3]:
df.query("date == '2010-01-01'")

,date,value,series
39,2010-01-01,4354363.000,USA
20,2010-01-01,4197204.000,China
39,2010-01-01,379640.425,UK


In [4]:
df['label'] = np.where(df.date == df.date.max(), df.series, '')

base = alt.Chart(df.query('date > "1990-01-01" and series != "UK"')).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q', title=''),
    color=alt.Color('series:N', title='Country', legend=None),
)

line = base.mark_line()

labels = base.mark_text(align='left', dx=5, dy=0, fontSize=12).encode(
    text='label',
)

chart = line + labels

chart = chart.properties(
    title=alt.TitleParams(
        text='Electricity Generation',
        subtitle=['Gigawatt hours, source: IEA via ECO-API', ''],
        anchor='start',
        fontSize=16,
        subtitleColor="rgb(143, 143, 143)",
        fontWeight='normal'
    ),
    width=400,
    height=300
)

chart.save("chn_usa.json")
chart.save("chn_usa.png", scale_factor=2.0)


chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [5]:
df.query("date == '2012-01-01'")    

,date,value,series,label
41,2012-01-01,4270884.000,USA,
22,2012-01-01,4984732.000,China,
41,2012-01-01,361087.495,UK,


# US vs China Renewables

In [17]:
us_rn = pd.read_json("https://api.economicsobservatory.com/usa/rGwh?vega")
us_nuc = pd.read_json("https://api.economicsobservatory.com/usa/nGwh?vega")
us_rn = us_rn.rename(columns={'value': 'us_rn'})
us_nuc = us_nuc.rename(columns={'value': 'us_nuc'})
cn_rn = pd.read_json("https://api.economicsobservatory.com/chn/rGwh?vega")
cn_nuc = pd.read_json("https://api.economicsobservatory.com/chn/nGwh?vega")
cn_rn = cn_rn.rename(columns={'value': 'cn'})
cn_nuc = cn_nuc.rename(columns={'value': 'cn_nuc'})
us_rn = us_rn.merge(us_nuc, on='date')
cn_rn = cn_rn.merge(cn_nuc, on='date')
df = pd.merge(us_rn, cn_rn, on='date')

In [19]:
df['us'] = df['us_rn'] + df['us_nuc']
df['cn'] = df['cn'] + df['cn_nuc']
df['ratio'] = df['us'] / df['cn']
df[['date', 'us', 'cn', 'ratio']]

,date,us,cn,ratio
0,1990-01-01,980830.000,126788.000,7.735984
1,1991-01-01,999541.000,125165.000,7.985787
2,1992-01-01,980286.000,132601.000,7.392750
3,1993-01-01,1002565.000,153566.000,6.528561
4,1994-01-01,1015378.000,182661.000,5.558811
5,1995-01-01,1098149.000,206495.000,5.318042
6,1996-01-01,1138079.000,203944.000,5.580350
7,1997-01-01,1069493.000,213340.000,5.013092
8,1998-01-01,1079683.000,224964.000,4.799359
9,1999-01-01,1123119.000,221787.000,5.063953


In [9]:
us_rn

,date,value
0,1971-01-01,264338.000
1,1972-01-01,272826.000
2,1973-01-01,268132.000
3,1974-01-01,306937.000
4,1975-01-01,306703.000
5,1976-01-01,290640.000
6,1977-01-01,226925.000
7,1978-01-01,286721.000
8,1979-01-01,287209.000
9,1980-01-01,284590.000


In [11]:
cn_rn

,date,value
0,1990-01-01,126788.000
1,1991-01-01,125165.000
2,1992-01-01,132601.000
3,1993-01-01,151962.000
4,1994-01-01,167905.000
5,1995-01-01,193662.000
6,1996-01-01,189606.000
7,1997-01-01,198922.000
8,1998-01-01,210864.000
9,1999-01-01,206838.000


# Renewables

In [6]:
total_gen = pd.read_json("https://api.economicsobservatory.com/gbr/eGWh?vega")
total_gen['series'] = 'Total'
renewables_gen = pd.read_json("https://api.economicsobservatory.com/gbr/rGwh?vega")
renewables_gen['series'] = 'Renewable'
df = pd.concat([total_gen, renewables_gen])

In [7]:
df['label'] = np.where(df.date == df.date.max(), df.series, '')

base = alt.Chart(df.query('date > "1990-01-01" and series != "UK"')).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q', title=''),
    color=alt.Color('series:N', title='Country', legend=None),
)

line = base.mark_line()

labels = base.mark_text(align='left', dx=5, dy=0, fontSize=12).encode(
    text='label',
)

chart = line + labels

chart = chart.properties(
    title=alt.TitleParams(
        text='Electricity Generation',
        subtitle=['Gigawatt hours, source: IEA via ECO-API', ''],
        anchor='start',
        fontSize=16,
        subtitleColor="rgb(143, 143, 143)",
        fontWeight='normal'
    ),
    width=400,
    height=300
)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

# 

# UK vs France

In [55]:
gbr_df = pd.read_json("https://api.economicsobservatory.com/usa/eGWh?vega")
gbr_df['series'] = 'UK'
fra_df = pd.read_json("https://api.economicsobservatory.com/fra/eGWh?vega")
fra_df['series'] = 'France'

df = pd.concat([gb_df, fra_df])
df

,date,value,series
0,1971-01-01,255756.000,UK
1,1972-01-01,262767.000,UK
2,1973-01-01,281352.000,UK
3,1974-01-01,272402.000,UK
4,1975-01-01,270790.000,UK
...,...,...,...
47,2018-01-01,576458.538,France
48,2019-01-01,566290.249,France
49,2020-01-01,527885.573,France
50,2021-01-01,550952.316,France


In [3]:
gbr_df = pd.read_json("https://api.economicsobservatory.com/usa/eGWh?vega")
gbr_df['series'] = 'UK'
fra_df = pd.read_json("https://api.economicsobservatory.com/deu/eGWh?vega")
fra_df['series'] = 'France'

df = pd.concat([gb_df, fra_df])
df

df['label'] = np.where(df.date == df.date.max(), df.series, '')

base = alt.Chart(df.query('date > "1990-01-01"')).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q', title=''),
    color=alt.Color('series:N', title='Country', legend=None),
)

line = base.mark_line()

labels = base.mark_text(align='left', dx=5, dy=0, fontSize=12).encode(
    text='label',
)

chart = line + labels

chart = chart.properties(
    title=alt.TitleParams(
        text='Electricity Generation',
        subtitle=['Gigawatt hours, source: IEA via ECO-API', ''],
        anchor='start',
        fontSize=16,
        subtitleColor="rgb(143, 143, 143)",
        fontWeight='normal'
    ),
    width=400,
    height=300
)

chart

NameError: name 'gb_df' is not defined

# UK vs France: Nuclear

In [13]:
gbr_df = pd.read_json("https://api.economicsobservatory.com/gbr/nGWh?vega")
gbr_df['series'] = 'UK'
fra_df = pd.read_json("https://api.economicsobservatory.com/fra/nGWh?vega")
fra_df['series'] = 'France'

df = pd.concat([gbr_df, fra_df])
df

df['label'] = np.where(df.date == df.date.max(), df.series, '')

base = alt.Chart(df.query('date > "1890-01-01" and series=="UK"')).encode(
    x=alt.X('date:T'),
    y=alt.Y('value:Q', title=''),
    color=alt.Color('series:N', title='Country', legend=None),
)

line = base.mark_line()

labels = base.mark_text(align='left', dx=5, dy=0, fontSize=12).encode(
    text='label',
)

chart = line + labels

chart = chart.properties(
    title=alt.TitleParams(
        text='Nuclear Power Generation',
        subtitle=['Gigawatt hours, source: IEA via ECO-API', ''],
        anchor='start',
        fontSize=16,
        subtitleColor="rgb(143, 143, 143)",
        fontWeight='normal'
    ),
    width=400,
    height=300
)

chart.save("nuclear_power.json")
chart.save("nuclear_power.png", scale_factor=2.0)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


In [10]:
df.query("series == 'UK' and value > 47723.100")

,date,value,series,label
12,1983-01-01,49928.000,UK,
13,1984-01-01,53979.000,UK,
14,1985-01-01,61095.000,UK,
15,1986-01-01,59079.000,UK,
16,1987-01-01,55238.000,UK,
17,1988-01-01,63456.000,UK,
18,1989-01-01,71734.000,UK,
19,1990-01-01,65749.000,UK,
20,1991-01-01,70543.000,UK,
21,1992-01-01,76807.000,UK,


In [51]:
df

,date,value,series
0,1971-01-01,255756.000,Total
1,1972-01-01,262767.000,Total
2,1973-01-01,281352.000,Total
3,1974-01-01,272402.000,Total
4,1975-01-01,270790.000,Total
...,...,...,...
47,2018-01-01,109771.864,Renewable
48,2019-01-01,119581.499,Renewable
49,2020-01-01,134740.952,Renewable
50,2021-01-01,122178.143,Renewable


In [48]:
df

,date,value
0,1971-01-01,255756.000
1,1972-01-01,262767.000
2,1973-01-01,281352.000
3,1974-01-01,272402.000
4,1975-01-01,270790.000
...,...,...
47,2018-01-01,109771.864
48,2019-01-01,119581.499
49,2020-01-01,134740.952
50,2021-01-01,122178.143


In [40]:
df[df.date == df.date.max()]

,date,value,series,label
51,2022-01-01,4479416.422,USA,USA
32,2022-01-01,8872191.256,China,China
